In [1]:
import os
cwd = os.getcwd()

In [2]:
cd {cwd}

/mnt/c/Users/hadis/Desktop/iust


# load model

In [10]:
import sys
sys.path.append("/home/hadi/iust/AP3D/")
import os
import time
import math
import os.path as osp
import numpy as np

import torch
from torch.utils.data import DataLoader


import models
import transforms.spatial_transforms as ST
import tools.data_manager as data_manager
from tools.video_loader import VideoDataset
from tools.utils import Logger
from tools.eval_metrics import evaluate
from commons import modify_model
from tqdm import tqdm

class Args:
    root = "/home/hadi/iust/datasets"
    dataset = "mars"
    workers = 4
    height = 256
    width = 128
    test_frames = 16
    arch = "ap3dres50"
    resume = "./"
    pretrain = "/home/hadi/iust/AP3D/logs/row41/best_model.pth.tar"
    test_epochs = [240]
    distance = "cosine"
    gpu = "0"
    
args = Args()

In [11]:

def extract(model, vids, use_gpu):
    n, c, f, h, w = vids.size()
    assert(n == 1)

    if use_gpu:
        feat = torch.cuda.FloatTensor()
    else:
        feat = torch.FloatTensor()
    for i in range(math.ceil(f/args.test_frames)):
        clip = vids[:, :, i*args.test_frames:(i+1)*args.test_frames, :, :]
        
        if use_gpu:
            clip = clip.cuda()
        output = model(clip)
        feat = torch.cat((feat, output), 1)

    feat = feat.mean(1)
    feat = model.bn(feat)
    feat = feat.data.cpu()

    return feat


def test(model, queryloader, galleryloader, use_gpu):
    since = time.time()
    model.eval()

    qf, q_pids, q_camids = [], [], []
    print('queryloader')
    for batch_idx, (vids, pids, camids) in tqdm(enumerate(queryloader)):
        if (batch_idx + 1) % 1000==0:
            print("{}/{}".format(batch_idx+1, len(queryloader)))

        qf.append(extract(model, vids, use_gpu).squeeze())
        q_pids.extend(pids)
        q_camids.extend(camids)

    qf = torch.stack(qf)
    q_pids = np.asarray(q_pids)
    q_camids = np.asarray(q_camids)
    print("Extracted features for query set, obtained {} matrix".format(qf.shape))
    
    print('galleryloader')
    gf, g_pids, g_camids = [], [], []
    for batch_idx, (vids, pids, camids) in tqdm(enumerate(galleryloader)):
        if (batch_idx + 1) % 1000==0:
            print("{}/{}".format(batch_idx+1, len(galleryloader)))

        gf.append(extract(model, vids, use_gpu).squeeze())
        g_pids.extend(pids)
        g_camids.extend(camids)
    gf = torch.stack(gf)
    g_pids = np.asarray(g_pids)
    g_camids = np.asarray(g_camids)

    if args.dataset == 'mars':
        # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
        gf = torch.cat((qf, gf), 0)
        g_pids = np.append(q_pids, g_pids)
        g_camids = np.append(q_camids, g_camids)

    print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

    time_elapsed = time.time() - since
    print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    print("Computing distance matrix")
    m, n = qf.size(0), gf.size(0)
    distmat = torch.zeros((m,n))

    if args.distance == 'euclidean':
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        for i in range(m):
            distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
    else:
        q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
        g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
        qf = qf.div(q_norm.expand_as(qf))
        gf = gf.div(g_norm.expand_as(gf))
        for i in range(m):
            distmat[i] = - torch.mm(qf[i:i+1], gf.t())
    distmat = distmat.numpy()

    print("Computing CMC and mAP")
    cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

    print("Results ----------")
    print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
    print("------------------")

    return cmc[0]


In [ ]:
# Data augmentation
dataset = data_manager.init_dataset(name=args.dataset, root=args.root)

spatial_transform_test = ST.Compose([
            ST.Scale((args.height, args.width), interpolation=3),
            ST.ToTensor(),
            ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
temporal_transform_test = None
use_gpu = torch.cuda.is_available()
pin_memory = True if use_gpu else False

queryloader = DataLoader(
    VideoDataset(dataset.query, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=1, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)

galleryloader = DataLoader(
    VideoDataset(dataset.gallery, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=1, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)

print("Initializing model: {}".format(args.arch))
model = models.init_model(name=args.arch, num_classes=dataset.num_train_pids)
print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
    
model = models.init_model(name=args.arch, num_classes=dataset.num_train_pids)
modify_model(model, args)

with torch.no_grad():
    test(model, queryloader, galleryloader, use_gpu)